In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

ModuleNotFoundError: No module named 'nltk'

In [2]:
# Load the dataset
df = pd.read_csv("dataset1.csv")

In [3]:
df.head()

,appid,review,sentiment
0,10,too shity it is incredible that this fucking s...,Negative
1,10,W game but bad on steam version nonsteam bette...,Negative
2,10,Why is this game so boring?! There are no bots...,Negative
3,10,game's peace is too slow and weapons are too i...,Negative
4,10,.,Negative


In [78]:
# Step 1: Preprocess Text Data
def preprocess_text(text):
    if isinstance(text, float):  # Handle NaN/float values
        text = ""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    words = word_tokenize(text)  # Tokenization
    return " ".join(words)  # Reconstruct text

# Apply preprocessing to dataset
df["cleaned_review"] = df["review"].fillna("").apply(preprocess_text)

In [80]:
# Remove rows with empty cleaned reviews
df = df[df["cleaned_review"].str.strip() != ""]

In [81]:
print("Number of reviews:", len(df))  # Should be greater than 0
print("Sample reviews:", df["cleaned_review"].head(5))  # Check sample text

Number of reviews: 10673
Sample reviews: 0    too shity it is incredible that this fucking s...
1    w game but bad on steam version nonsteam bette...
2    why is this game so boring there are no bots i...
3    games peace is too slow and weapons are too in...
5    the game is great but standstill crouching wit...
Name: cleaned_review, dtype: object


In [82]:

# Debugging checks
print("Dataset size after cleaning:", df.shape)
print("Sample cleaned reviews:\n", df["cleaned_review"].head())

Dataset size after cleaning: (10673, 4)
Sample cleaned reviews:
 0    too shity it is incredible that this fucking s...
1    w game but bad on steam version nonsteam bette...
2    why is this game so boring there are no bots i...
3    games peace is too slow and weapons are too in...
5    the game is great but standstill crouching wit...
Name: cleaned_review, dtype: object


In [83]:
# Strip any extra spaces that might be causing issues with mapping
df["sentiment"] = df["sentiment"].str.strip()

In [84]:
# Save preprocessed data
df.to_csv("preprocessed_dataset1.csv", index=False)
print("Preprocessing complete!")

Preprocessing complete!


In [85]:
# Step 2: Convert Sentiment Labels to Numeric
df["sentiment"] = df["sentiment"].map({"Positive": 1, "Negative": 0})

In [88]:
# Check the unique values again
print(df["sentiment"].unique())

[0 1]


In [87]:
df = df.dropna(subset=['sentiment'])

In [75]:
#redo


In [65]:
# Convert labels to integer
y = df["sentiment"].astype(int)

In [66]:
X = df["cleaned_review"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [67]:
# Step 4: TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features to avoid sparse matrix issues
X_train = vectorizer.fit_transform(X_train)  
X_test = vectorizer.transform(X_test)

In [68]:
# Debugging checks
print("Shape of X_train:", X_train.shape)  # Should match number of rows in train_df
print("Shape of X_test:", X_test.shape)    # Should match number of rows in test_df
print("Length of y_train:", len(y_train))
print("Length of y_test:", len(y_test))

Shape of X_train: (8538, 5000)
Shape of X_test: (2135, 5000)
Length of y_train: 8538
Length of y_test: 2135


In [69]:
# Step 5: Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=20, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=300, random_state=42)

In [70]:
# Step 6: Evaluate Model
y_pred = rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("\n🔹 Model Accuracy:", accuracy)
print("\n🔹 Classification Report:\n", classification_report(y_test, y_pred))


🔹 Model Accuracy: 0.863231850117096

🔹 Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.01      0.02       295
           1       0.86      1.00      0.93      1840

    accuracy                           0.86      2135
   macro avg       0.93      0.51      0.47      2135
weighted avg       0.88      0.86      0.80      2135



In [89]:
from sklearn.model_selection import train_test_split

# Step 1: Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(df["cleaned_review"], df["sentiment"], test_size=0.2, random_state=42)

# Step 2: Create new DataFrames for training and testing
train_data = pd.DataFrame({"review": X_train, "sentiment": y_train})
test_data = pd.DataFrame({"review": X_test, "sentiment": y_test})

# Step 3: Save the DataFrames to CSV files
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

# Check the saved files
print("Train data and Test data have been saved to 'train_data.csv' and 'test_data.csv'.")

Train data and Test data have been saved to 'train_data.csv' and 'test_data.csv'.


# Count the number of positive and negative in train dataset
train_sentiment_counts = train_data['sentiment'].value_counts()

# Count the number of positive and negative in test dataset
test_sentiment_counts = test_data['sentiment'].value_counts()

# Print the counts
print("Train Data Sentiment Counts:")
print(train_sentiment_counts)

print("\nTest Data Sentiment Counts:")
print(test_sentiment_counts)

In [90]:
train_data = pd.read_csv("train_data.csv")
test_data = pd.read_csv("test_data.csv")

# Ensure that 'cleaned_review' and 'sentiment' columns exist
X_train = train_data['review']
y_train = train_data['sentiment']
X_test = test_data['review']
y_test = test_data['sentiment']

In [91]:
# Step 2: TF-IDF Vectorization
vectorizer = TfidfVectorizer(max_features=5000)  # Limit features to avoid sparse matrix issues
X_train_tfidf = vectorizer.fit_transform(X_train)  # Fit and transform train data
X_test_tfidf = vectorizer.transform(X_test)  # Transform test data (without fitting)

In [92]:
# Debugging checks
print("Shape of X_train_tfidf:", X_train_tfidf.shape)  # Should match number of rows in train_data
print("Shape of X_test_tfidf:", X_test_tfidf.shape)    # Should match number of rows in test_data
print("Length of y_train:", len(y_train))
print("Length of y_test:", len(y_test))



Shape of X_train_tfidf: (8538, 5000)
Shape of X_test_tfidf: (2135, 5000)
Length of y_train: 8538
Length of y_test: 2135


In [93]:
# Step 3: Train Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=300, max_depth=20, random_state=42)
rf_model.fit(X_train_tfidf, y_train)


RandomForestClassifier(max_depth=20, n_estimators=300, random_state=42)

In [94]:
# Step 4: Evaluate Model
y_pred = rf_model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("\n🔹 Model Accuracy:", accuracy)
print("\n🔹 Classification Report:\n", classification_report(y_test, y_pred))


🔹 Model Accuracy: 0.863231850117096

🔹 Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.01      0.02       295
           1       0.86      1.00      0.93      1840

    accuracy                           0.86      2135
   macro avg       0.93      0.51      0.47      2135
weighted avg       0.88      0.86      0.80      2135

